In [1]:
import requests
import json
import base64
import os
import time
import pandas as pd

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [9]:
config_filename = 'base_login_root_wel.json'

base_authentication_uri = 'api/authentication/1.0/Session/'
base_sstore_uri = 'api/com.oracle.solaris.rad.sstore/1.0/'
sstore_namespace_uri = '{}Namespace/'.format(base_sstore_uri)

In [4]:
if config_filename:
    with open(config_filename, 'r') as f:
        server_connection_info = json.load(f)
        
server_connection_info

{'server_name': 't8ldom3.us.oracle.com',
 'server_port': '6788',
 'cert_loaction': 'certs/hostca_10.134.74.33.crt',
 'config_json': {'username': 'root',
  'password': 'Welcome1',
  'scheme': 'pam',
  'preserve': True,
  'timeout': -1}}

In [5]:
def rad_rest_login(session_name, server_connection_info, base_authentication_uri):
    login_url = 'https://{0}:{1}/{2}'.format(*list(map(server_connection_info.get, ['server_name', 'server_port'])), base_authentication_uri)
    print("Logging in with this URL: " + login_url)

    try:
        response = session_name.post(login_url, json = server_connection_info['config_json'], verify = server_connection_info['cert_loaction'])
    except:
        print('no connection')
        response = 'empty'
    
    return response

In [6]:
def rad_rest_request(request_type, session_name, server_connection_info, rad_rest_uri, payload = None):
    
    # Create a list of variables
    query_vars = list(map(server_connection_info.get, ['server_name', 'server_port']))
    query_vars.append(rad_rest_uri)

    # Create query url to be used
    query_url = 'https://{0}:{1}/{2}'.format(*query_vars)

    response = session_name.request(request_type, query_url, json = payload)
    
    return response

In [7]:
s = requests.Session()

login_answer = rad_rest_login(s, server_connection_info, base_authentication_uri)

# printing the answers coming back to show the proces.
print(login_answer.status_code)
print(login_answer.text)

Logging in with this URL: https://t8ldom3.us.oracle.com:6788/api/authentication/1.0/Session/
201
{
        "status": "success",
        "payload": {
                "href": "/api/com.oracle.solaris.rad.authentication/1.0/Session/_rad_reference/7424"
        }
}


In [22]:
json_body = {
    'ssids': ['//:*'],
    'range': {
              'range_type': 'RANGE_BY_TIME',
              'range_by_time': {
                                'start_ts': -1,
                                'end_ts': -1,
                                'step': 1
                               },   
              'show_unstable': True,
              'show_unbrowsable': True 
             }    
}

In [31]:
print('{}_rad_method/read'.format(sstore_namespace_uri))
query_answer = rad_rest_request('PUT', s, server_connection_info, '{}_rad_method/read'.format(sstore_namespace_uri), json_body)
query_text = json.loads(query_answer.text)

print(query_answer.status_code, '\n')
# print(json.dumps(query_text, indent = 2))
query_df = pd.DataFrame(query_text['payload']['records'])

api/com.oracle.solaris.rad.sstore/1.0/Namespace/_rad_method/read
200 



In [29]:
query_df['ssid']

0                      //:class.app/oracle/rdbms
1                    //:class.app/solaris/apache
2               //:class.app/solaris/audit/class
3     //:class.app/solaris/compliance/assessment
4           //:class.app/solaris/compliance/rule
5      //:class.app/solaris/install/auto-install
6                      //:class.app/solaris/nscd
7                       //:class.app/solaris/smf
8                    //:class.app/solaris/sstore
9                //:class.app/solaris/sysstat/fs
10            //:class.app/solaris/sysstat/netfs
11          //:class.app/solaris/sysstat/sysconf
12            //:class.app/solaris/sysstat/zpool
13                                 //:class.chip
14                           //:class.collection
15                                 //:class.core
16                                  //:class.cpu
17                                  //:class.dax
18                                  //:class.dev
19                              //:class.devices
20                  

In [32]:
json_body = {
    'ssids': ['//:class.app/solaris/compliance/assessment//:*'],
    'range': {
              'range_type': 'RANGE_BY_TIME',
              'range_by_time': {
                                'start_ts': -1,
                                'end_ts': -1,
                                'step': 1
                               },   
              'show_unstable': True,
              'show_unbrowsable': True 
             }    
}

In [35]:
print('{}_rad_method/read'.format(sstore_namespace_uri))
query_answer = rad_rest_request('PUT', s, server_connection_info, '{}_rad_method/read'.format(sstore_namespace_uri), json_body)
query_text = json.loads(query_answer.text)

print(query_answer.status_code, '\n')
print(json.dumps(query_text, indent = 2))
query_df = pd.DataFrame(query_text['payload']['records'])

api/com.oracle.solaris.rad.sstore/1.0/Namespace/_rad_method/read
200 

{
  "status": "success",
  "payload": {
    "records": [
      {
        "ssid": "//:class.app/solaris/compliance/assessment//:stat.results",
        "points": [
          {
            "point_type": "RANGE_POINT",
            "point_value": null,
            "point_range": {
              "start_ts": 0,
              "end_ts": 0
            }
          }
        ]
      },
      {
        "ssid": "//:class.app/solaris/compliance/assessment//:stat.score",
        "points": [
          {
            "point_type": "RANGE_POINT",
            "point_value": null,
            "point_range": {
              "start_ts": 0,
              "end_ts": 0
            }
          }
        ]
      },
      {
        "ssid": "//:class.app/solaris/compliance/assessment//:stat.uuid",
        "points": [
          {
            "point_type": "RANGE_POINT",
            "point_value": null,
            "point_range": {
              "st

In [34]:
query_df['ssid']

0    //:class.app/solaris/compliance/assessment//:s...
1    //:class.app/solaris/compliance/assessment//:s...
2    //:class.app/solaris/compliance/assessment//:s...
Name: ssid, dtype: object

In [36]:
from datetime import datetime

In [40]:
ts = int("1588080480000000")
# if you encounter a "year is out of range" error the timestamp
# may be in milliseconds, try `ts /= 1000` in that case
print(datetime.utcfromtimestamp(ts/1000000).strftime('%Y-%m-%d %H:%M:%S'))

2020-04-28 13:28:00


In [65]:
from datetime import timezone
dt = datetime(2015, 10, 19, 20, 31)
timestamp = int((dt - datetime(1970, 1, 1)).total_seconds())*1000000
print(timestamp)

1445286660000000


In [67]:
dt.strftime('%Y-%m-%d %H:%M:%S')

'2015-10-19 20:31:00'

In [48]:
from datetime import timezone
dt = datetime(2015, 10, 19)
# timestamp = dt.timestamp()
timestamp = dt.replace(tzinfo=timezone.utc).timestamp()
print(timestamp)

1445212800.0


In [60]:
dir((dt - datetime(1970, 1, 1)).total_seconds())

['__abs__',
 '__add__',
 '__bool__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__divmod__',
 '__doc__',
 '__eq__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getformat__',
 '__getnewargs__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__le__',
 '__lt__',
 '__mod__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__pos__',
 '__pow__',
 '__radd__',
 '__rdivmod__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rfloordiv__',
 '__rmod__',
 '__rmul__',
 '__round__',
 '__rpow__',
 '__rsub__',
 '__rtruediv__',
 '__set_format__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__truediv__',
 '__trunc__',
 'as_integer_ratio',
 'conjugate',
 'fromhex',
 'hex',
 'imag',
 'is_integer',
 'real']